# Function Dependency Equation for Free Product of 2 Cyclic Groups.

Consider $G=\langle x| x^m=1 \rangle * \langle y| y^n=1 \rangle$, where $m,n \in \mathbb{Z}_{\ge0}\setminus \{1\}$. Note that $|G|<\infty \iff mn>0$. 
Take the generating set $S=S_x\cup S_y$ for $G$, where 
$$S_x=\{x\}\cup \iota(m=0)\cdot\{x^{-1}\}, S_y=\{y\}\cup \iota(n=0)\cdot\{y^{-1}\},$$ 
and $$\iota(\mathcal{P})=\begin{cases}\{1\},&\mathcal{P}\\ \emptyset,& \lnot \mathcal{P}\end{cases}$$ with $\cdot$ denoting elementwise group multiplication on sets [$A\cdot B = \{ab: a\in A,\ b\in B\}$]. 

For $g\in G$ and $X\subseteq G$, let $F_{g,X}(t)$ be series for the set of all words in $S^*$ equivalent to $g\in G$, with proper nonempty prefixes avoiding $X$, characterized by the word length. We set up functional equations involving $F_{g,X}$, for which $\{g\}\cup X\subseteq \{v^k: k\in \mathbb{Z}\}$, and $v\in \{x,y\}$. Finally, we solve for $F(t):=F_{1,\emptyset}(t)$.

In [1]:
# choose your desired values for m,n
# for now, only work for finite G, (mn>0)
m = 2
n = 3

# Do we want word concatenation to reduce to group multiplication?
AUTO_REDUCE = True
DEFAULT_RS = None # a rewiting system in order to make group element reduction possible, will be set later


In [2]:
# need to define set-set, set-element multiplication
def set_mult(A,B):
    # turn any non-set object into a singleton set containing that object
    A,B = map(lambda S: S if S in Sets else Set([S]),[A,B])
    C= Set([a*b for a,b in cartesian_product([A,B])]) # assuming element multiplication is well defined
    if AUTO_REDUCE and DEFAULT_RS is not None:
        C = Set([DEFAULT_RS.reduce(c) for c in C])
    return C

# Let's overload the '*' operator for Set objects
# Set is a function, but returns an instance of some spectifed kind of Set class, which is accessible via .parent()
reverse_if = lambda seq, cond: list(seq) if not cond else list(reversed(seq))
Set().parent()._mul_ =  lambda self, other, switch_sides=False: set_mult(*(reverse_if([self,other], switch_sides)))

In [3]:
# compute the free product
F2.<x,y> = FreeGroup()
G = F2/ (x^m,y^n)
x,y = G.gens()

# need to write words in reduced form
Grs = G.rewriting_system()
DEFAULT_RS = Grs

In [4]:
# test it out
A = Set([x,y,G.one()])

print(x*A) # with reduction

# very simple to supress/invoke group reduction
AUTO_REDUCE = False
print(x*A) # without reduction
AUTO_REDUCE = True
print(x*A) # with reduction

{x*y, 1, x}
{x*y, x^2, x}
{x*y, 1, x}


In [5]:
# latex expression for sets
def set_to_latex(A):
    if A == Set():
        return r'\emptyset'
    return latex(A)

In [6]:
# define a space of variables indexed by the (g,X) pairs
# start with what we wish to solve, (1,\emptyset).
def series_expression(g,X):
    return 'F_{%s,%s}(t)'%(latex(g),set_to_latex(X))
var_space = {}
goal_pair = (G.one(), Set())
VAR_COUNTER = 0 # increment every time a new series variable is defined

def add_series_var(pair):
    if pair in var_space:
        return False
    global VAR_COUNTER
    g,X = pair
    var_space[pair] = var('v%d'%VAR_COUNTER, latex_name = series_expression(g,X))
    VAR_COUNTER += 1
    return True

add_series_var(goal_pair)
    

True

In [7]:
pretty_print(var_space)

{(1, {}): v0}

In [8]:
# Now get the cyclic factors

# Sage is not good with subgroups
# Gx = G.subgroup((x,))
# Gy = G.subgroup((y,))
# xx,yy = Gx.gens()+Gy.gens()

def in_cyclic_factor(el, v):
    '''
        returns if the group element is in the given cyclic factor
        el: the element in G to check
        v: either x or y, the generator of the factor 
    '''
    expr = el.syllables()
    if not expr:
        return True
    return len(expr)==1 and expr[0][0] == v
    
def subset_of_cyclic_factor(A,v):
    # assume finite sets only
    return all((in_cyclic_factor(el,v) for el in A))

In [9]:
# for finite cyclic factors, we don't want negative exponents. We redefine the __pow__ method.
if AUTO_REDUCE and DEFAULT_RS is not None:
    if '_old_pow' not in vars():
        _old_pow = type(x).__pow__
    def _new_pow(a, k):
        if k not in ZZ or k>=0:
            # case isn't broken, don't change it
            return _old_pow(a,k)

        # k is a negative integer
        ans = _old_pow(a,k)
    #     if AUTO_REDUCE:
    #         ans = DEFAULT_RS.reduce(ans)
        nans = G.one()
        for v,ep in ans.syllables():
            modval = m if v==x else n
            nans*= _old_pow(v, ep if modval==0 else ep%modval)
        return nans
    type(x).__pow__ = _new_pow

In [10]:
# For multiplication, want auto reduation to be available
# if AUTO_REDUCE and DEFAULT_RS is not None:
#     if '_old_mul' not in vars():
#         _old_mul = type(x).__mul__
#     def _new_mul(a, b):
#         return DEFAULT_RS.reduce(_old_mul(a,b))
#     type(x).__mul__ = _new_mul


In [11]:
x,x^2,x^3*x,x^-1,(x*y)^-1, x^-2,y^-3,(x*y^3)^-1, '_old_pow' in vars()

(x, x^2, x^4, x, y^2*x, 1, 1, x, True)

In [12]:
var('t')

t

In [13]:
# construct the functional equation given the desired (g,X) pair
# assume {g}\cup X is completely in one of the cyclic factors
# CURRENTLY FINITE CASE ONLY

def get_FD_equation(pair, ret_new_pairs = True):
    '''
        returns a functional depency equation
        if ret_new_pairs is set to True, return a (equation, set of (g,X) pairs) pair instead
    '''
    if ret_new_pairs:
        newpairs = set()
    g,X = pair
    eqn = None
    one = G.one()
    oneset = Set([one])
    if one not in X:
        XU1 = X.union(oneset)
        if add_series_var((one,X)) and ret_new_pairs:
            newpairs.add((one,X))
        if add_series_var((g,XU1)) and ret_new_pairs:
            newpairs.add((g,XU1))
        if g==one:
            eqn = 1+var_space[(one,X)]*(var_space[(one,XU1)]-1)
        else:
            eqn = var_space[(one,X)]*var_space[(g,XU1)]
    elif subset_of_cyclic_factor(X,x):
        if g!=one:
            eqn = t*ZZ(g==x)
            if x not in X:
                [g] = x^-1*Set([g]) # use set so auto reduction is implemented
                X = x^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
        else:
            set_yinv = Set([y^-1]) 
            if add_series_var((y^-1,set_yinv)) and ret_new_pairs:
                newpairs.add((y^-1,set_yinv))
            eqn = 1+t*var_space[(y^-1,set_yinv)]
            if x not in X:
                g = x^-1
                X = x^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
    else:
        # the y factor case
        if g!=one:
            eqn = t*ZZ(g==y)
            if y not in X:
                [g] = y^-1*Set([g])
                X = y^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
        else:
            set_xinv = Set([x^-1]) 
            if add_series_var((x^-1,set_xinv)) and ret_new_pairs:
                newpairs.add((x^-1,set_xinv))
            eqn = 1+t*var_space[(x^-1,set_xinv)]
            if x not in X:
                g = y^-1
                X = y^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
    eqn = var_space[pair]==eqn
    return (eqn,newpairs) if ret_new_pairs else eqn
    


In [14]:
# Let us build the system of equations
system = []
queue = [goal_pair]

# use BFS
while queue:
    pair = queue.pop(0)
    eqn, newp = get_FD_equation(pair)
    queue.extend(newp)
    system.append(eqn)


In [15]:
# Here are the equations
for eqn in system:
    print(latex(eqn),r'\\')
print()
print()
for eqn in system:
    print(eqn)
    pretty_print(eqn)

{F_{1,\emptyset}(t)} = {F_{1,\emptyset}(t)} {\left({F_{1,\left\{1\right\}}(t)} - 1\right)} + 1 \\
{F_{1,\left\{1\right\}}(t)} = t {F_{y^{2},\left\{y^{2}\right\}}(t)} + t {F_{x,\left\{x\right\}}(t)} + 1 \\
{F_{x,\left\{x\right\}}(t)} = {F_{1,\left\{x\right\}}(t)} {F_{x,\left\{1, x\right\}}(t)} \\
{F_{y^{2},\left\{y^{2}\right\}}(t)} = {F_{1,\left\{y^{2}\right\}}(t)} {F_{y^{2},\left\{1, y^{2}\right\}}(t)} \\
{F_{1,\left\{x\right\}}(t)} = {F_{1,\left\{x\right\}}(t)} {\left({F_{1,\left\{1, x\right\}}(t)} - 1\right)} + 1 \\
{F_{x,\left\{1, x\right\}}(t)} = t \\
{F_{1,\left\{y^{2}\right\}}(t)} = {F_{1,\left\{y^{2}\right\}}(t)} {\left({F_{1,\left\{1, y^{2}\right\}}(t)} - 1\right)} + 1 \\
{F_{y^{2},\left\{1, y^{2}\right\}}(t)} = t {F_{y,\left\{y, y^{2}\right\}}(t)} \\
{F_{1,\left\{1, x\right\}}(t)} = t {F_{y^{2},\left\{y^{2}\right\}}(t)} + 1 \\
{F_{1,\left\{1, y^{2}\right\}}(t)} = t {F_{y^{2},\left\{y, y^{2}\right\}}(t)} + t {F_{x,\left\{x\right\}}(t)} + 1 \\
{F_{y,\left\{y, y^{2}\right\}}(t)} 

v0 == v0*(v1 - 1) + 1

v1 == t*v2 + t*v3 + 1


v1 == t*v2 + t*v3 + 1

v3 == v4*v5


...

v12 == v12*(v15 - 1) + 1

v14 == 0


v14 == 0

v15 == t*v14 + t*v3 + 1


v15 == t*v14 + t*v3 + 1

In [16]:
# Finally solve the system

# reports an empty list
# solve(system, *var_space.values(), solution_dict=True)

In [17]:
# first, isolate variables on the LHS

for i, eqn in enumerate(system):
    system[i] = solve(eqn,eqn.lhs())[0]
    

In [18]:
for eqn in system:
    pretty_print(eqn)

v0 == -1/(v1 - 2)

v1 == t*v2 + t*v3 + 1

v3 == v4*v5

v2 == v6*v7

v4 == -1/(v8 - 2)

...

v11 == v12*v14

v13 == t

v12 == -1/(v15 - 2)

v14 == 0

v15 == t*v14 + t*v3 + 1

In [19]:
# now make some substitutions

change = 1
numit = 0

while change:
    change = 0
    # build a dependency graph
    eq_by_lhs = {eq.lhs():eq for eq in system}
    dep_gr = {eq.lhs():set(eq.rhs().args())-{t} for eq in system}
    vars_det = [v for v,args in dep_gr.items() if not args] # the completely solved vars

    rev_dep_gr = {v:set() for v in eq_by_lhs} # diagraph by reversing the arcs of dep_gr
    for v,ag in dep_gr.items():
        for w in ag:
            rev_dep_gr[w].add(v)

    # apply BFS to make the substitutions possible
    vis = set(vars_det)
    queue = list(vis)
    done = set()
    while queue:
        nxtvar = queue.pop(0)
        # check if all RHS variables are visited before
        sub_dict = {}
        for dps in dep_gr[nxtvar]:
            if dps not in done:
                continue # can't substitute, a RHS varible is not yet reduced
            sub_dict[dps] = eq_by_lhs[dps].rhs()
            change = 1
        eq_by_lhs[nxtvar] = eq_by_lhs[nxtvar].subs(sub_dict)
        for pt in rev_dep_gr[nxtvar]:
            if pt not in vis:
                queue.append(pt)
                vis.add(pt)
        done.add(nxtvar)
    
    for i, eq in enumerate(system):
        system[i] = eq_by_lhs[eq.lhs()]
    numit+=1

print('Number of iteration taken to substitute',numit)
        


Number of iteration taken to substitute 2


In [20]:
for eq in eq_by_lhs.values():
    pretty_print(eq)

v0 == -1/(t^2*v4 + t*v2 - 1)

v1 == t^2*v4 + t*v2 + 1

v3 == t*v4

v2 == -t^2*v12/(t^2*v4 - 1)

v4 == (1/(t^3*v12/(t^2*v4 - 1) + 1))

...

v11 == 0

v13 == t

v12 == -1/(t*v3 - 1)

v14 == 0

v15 == t*v3 + 1

In [21]:
# get rid of every irrelevant variable
start = var_space[goal_pair]
eq_by_lhs = {eq.lhs():eq for eq in system}
dep_gr = {eq.lhs():set(eq.rhs().args())-{t} for eq in system}

comp_set = {start}
comp = [start]
ind = 0
while ind<len(comp):
    for v in dep_gr[comp[ind]]:
        if v not in comp_set:
            comp.append(v)
            comp_set.add(v)
    ind += 1

system = [eq for eq in system if eq.lhs() in comp_set]

In [22]:
pretty_print(comp)
for eq in system:
    print(eq)
    pretty_print(eq)

[v0, v4, v2, v12, v3]

v0 == -1/(t^2*v4 + t*v2 - 1)


v0 == -1/(t^2*v4 + t*v2 - 1)

v3 == t*v4


v3 == t*v4

...

v2 == -t^2*v12/(t^2*v4 - 1)

v4 == (1/(t^3*v12/(t^2*v4 - 1) + 1))


v4 == (1/(t^3*v12/(t^2*v4 - 1) + 1))

v12 == -1/(t*v3 - 1)


v12 == -1/(t*v3 - 1)

In [23]:
# make every equation eplicit. delete loops
for i, eqn in enumerate(system):
    system[i] = solve(eqn,eqn.lhs())[0]

In [24]:
for eq in system:
    print(eq)
    pretty_print(eq)

v0 == -1/(t^2*v4 + t*v2 - 1)


v0 == -1/(t^2*v4 + t*v2 - 1)

v3 == t*v4


v3 == t*v4

v2 == -t^2*v12/(t^2*v4 - 1)


v2 == -t^2*v12/(t^2*v4 - 1)

v4 == -1/2*(t^3*v12 - t^2 + sqrt(t^6*v12^2 + t^4 - 2*t^2 - 2*(t^5 + t^3)*v12 + 1) - 1)/t^2


v4 == -1/2*(t^3*v12 - t^2 + sqrt(t^6*v12^2 + t^4 - 2*t^2 - 2*(t^5 + t^3)*v12 + 1) - 1)/t^2

v12 == -1/(t*v3 - 1)


v12 == -1/(t*v3 - 1)

In [38]:
pretty_print(maxima.eliminate(system,[v12]))

[(-_SAGE_VAR_v2*(_SAGE_VAR_t^3*_SAGE_VAR_v3-_SAGE_VAR_t^2)*_SAGE_VAR_v4)-_SAGE_VAR_v2*(1-_SAGE_VAR_t*_SAGE_VAR_v3)+_SAGE_VAR_t^2,-_SAGE_VAR_t^2*((_SAGE_VAR_t^3*_SAGE_VAR_v2-2*_SAGE_VAR_t^2)*_SAGE_VAR_v4-_SAGE_VAR_t*_SAGE_VAR_v2-sqrt(_SAGE_VAR_t^6*_SAGE_VAR_v12^2-2*(_SAGE_VAR_t^5+_SAGE_VAR_t^3)*_SAGE_VAR_v12+_SAGE_VAR_t^4-2*_SAGE_VAR_t^2+1)+_SAGE_VAR_t^2+1),_SAGE_VAR_t^2*_SAGE_VAR_v0*_SAGE_VAR_v4+_SAGE_VAR_t*_SAGE_VAR_v0*_SAGE_VAR_v2-_SAGE_VAR_v0+1,_SAGE_VAR_v3-_SAGE_VAR_t*_SAGE_VAR_v4]

In [34]:
list(comp_set-{start})

[v12, v4, v3, v2]

In [ ]:
# scrap!!!
eqn = t^6*v0^4-3*t^4*v0^3-t^2*(t^2-3)*v0^2+(t^3-2*t^2-1)*v0-1
pretty_print(eqn)
ans=(solve([eqn==0],v0)[0]).rhs()

pretty_print(ans)